Ce notebook contient l'analyse faite sur l'entièreté des 150 données. L'analyse a ici pris en compte les points obtenus par question.

# Set Up

## Importation des différentes librairies

**Panda.** Librairie permettant de manipuler des tableaux de données avec des étiquettes de variables (colonnes, i.e. pour nous les questions) et d'individus (lignes, i.e. les formulaires ayant été rempli).

**Numpy.** Librairie permettant de faire des calculs scientifique fournissant des fonctions ainsi que des structures de données très performantes de calcul.

**Matplotlib.** Librairie permettant de créer des visuels statiques, animés et interactifs

**Seaborn.** Librairie permettant de visualiser des données, et fournit des interfaces de haut niveau pour dessiner des graphiques statistiques attrayants et informatifs.

In [73]:
import pandas
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics 
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier, OutputCodeClassifier

## Importation du fichier excel et manipulation

Commençons par importer le fichier excel comportant les questions avec réponses selon l'entreprise. Le tableau contenu dans ce fichier sera notre data frame. 

**Définition.** *Data frame.* Un Data frame est une structure de données bidimensionnelle, i.e. les données sont alignées de façon tabulaire en lignes
 et en colonnes.

In [74]:
data_frame = pandas.read_excel('../excel/question.xlsx', sheet_name = 'risk')

Vérifions que l'on a bien 150 lignes, i.e. 150 entreprises, et 18 colonnes, i.e. 16 questions, le total et le nom de l'entreprise.

In [75]:
data_frame.shape

(150, 18)

Retirons les colonnes ne nous intéressant pas, ou contenant des chaines de caractères. Nous ne travaillerons en  effet qu'avec des valeurs numériques.

In [76]:
data_frame = data_frame.drop(columns = ["Unnamed: 0"])

Divisons les réponses de la première question qui sont du type *<span style="color: LightPink;">nb - un secteur</span>*, et ne récupérons que la première partie, i.e. la valeur numérique *<span style="color: LightPink;">nb</span>*.

On affichera finalement le data frame.

In [77]:
data_frame['Q1'] = data_frame['Q1'].str.split(' - ').str[0].astype(int)

data_frame

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Total
0,13,9,2,0,0,2,7,3,0,0,0,0,0,1,0,0,24
1,8,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,4
2,13,9,2,0,0,2,7,3,0,0,1,0,1,1,0,0,26
3,13,9,0,0,0,2,7,3,0,0,0,0,0,1,0,1,23
4,6,9,4,0,1,2,7,3,0,4,0,0,0,1,0,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,4,9,0,1,1,2,7,3,0,0,0,0,1,3,0,0,27
146,5,18,0,0,0,2,7,3,0,4,1,1,1,0,0,0,37
147,8,4,2,4,0,0,7,0,1,4,1,1,1,3,1,0,29
148,3,9,4,4,0,0,0,0,0,0,1,1,0,1,0,0,20


Convertissons maintenant la valeur totale en une valeur numérique permettant de représenter le risque. Nous aurons ainsi : 
<ul>
    <li>Low : 0 - représentera les totaux compris entre 0 et 10,</li>
    <li>Medium : 1 - représentera les totaux compris entre 11 et 18,</li>
    <li>High : 2 - représentera les totaux compris entre 19 et 24,</li>
    <li>Critical : 3 - représentera les totaux compris entre 25 et 52.</li>
</ul>

Notons que :

**intervals** représente les intervalles pour lesquelles les totaux doivents correspondre selmon le risque 

**representation** représente l'affiliation du niveau de risque à une valeur numérique

In [78]:
intervals = [0, 10, 18, 24, 52]
representation = [0, 1, 2, 3]
data_frame['Total'] = pandas.cut(data_frame['Total'], bins = intervals, labels = representation)

data_frame['Total'] = data_frame['Total'].astype(int)

data_frame

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Total
0,13,9,2,0,0,2,7,3,0,0,0,0,0,1,0,0,2
1,8,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0
2,13,9,2,0,0,2,7,3,0,0,1,0,1,1,0,0,3
3,13,9,0,0,0,2,7,3,0,0,0,0,0,1,0,1,2
4,6,9,4,0,1,2,7,3,0,4,0,0,0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,4,9,0,1,1,2,7,3,0,0,0,0,1,3,0,0,3
146,5,18,0,0,0,2,7,3,0,4,1,1,1,0,0,0,3
147,8,4,2,4,0,0,7,0,1,4,1,1,1,3,1,0,3
148,3,9,4,4,0,0,0,0,0,0,1,1,0,1,0,0,2


Assurons nous qu'aucune ligne n'est nulle.

In [79]:
data_frame.isnull().sum()

Q1       0
Q2       0
Q3       0
Q4       0
Q5       0
Q6       0
Q7       0
Q8       0
Q9       0
Q10      0
Q11      0
Q12      0
Q13      0
Q14      0
Q15      0
Q16      0
Total    0
dtype: int64

# Machine Learning

Introduisons deux variables, notées *predicted_column* et *training_columns*, représentant respectivement la colonne total, et l'ensemble des questions. 

C'est ainsi, que nous retirerons la colonne Total du data frame, et que nous affecterons à notre variable *predicted_column*. Nous affecterons alors le data frame à la variable *training_columns* sans la colonne Total

In [80]:
training_columns = data_frame.drop(columns = ["Total"])
predicted_column = data_frame['Total']

Pour l'ensemble des modèles j'ai décidé de suivre le principe du 80 - 20 plutôt que le 60 - 40 proposé dans le document envoyé.

Séparons et générons nos différents ensembles d'entrainements et ensembles de tests.

À noter que le *random_state* permet de controler la part aléatoire appliquée aux données avant de pouvoir séparer et générer nos ensemble d'entrainement et de test. 

**Notation.** x représente l'axe des abscisses, donc les inputs, et y représente l'axe des ordonnées donc les outputs. De manière informelle, x est le data frame contenant l'ensemble de questions tandis que y est la colonne des niveaux de risques.

<ul>
    <li>La variable x_train représente la variable input d'entrainement, i.e. les questions qui serviront d'entrainement aux modèles.</li>
    <li>La variable y_train représente la variable output d'entrainement, i.e. les totaux qui serviront de confrimation aux modèles et associés à x_train.</li>
    <li>La variable x_test représente la variable input de test, i.e. les questions qui serviront de test aux modèles.</li>
    <li>La variable x_train représente la variable output de test, i.e. les totaux qui serviront de test aux modèles.</li>
</ul>

Pluis précisément, les variables de test seront les ensemble sur lesquels on testera les entrainements fait par nos modèles.


In [81]:
x_train, x_test, y_train, y_test = train_test_split(training_columns, predicted_column, test_size=0.4, random_state=100)

## Modèle 1 - Classification naïve bayésienne

Nous commençons avec classification naïve bayésienne, i.e. basée sur le théorème de Bayes : Soit $A,B$ deux évènements. Si $\mathbb P(B) \neq 0$, 

$$ \mathbb P_B (A) \,\, = \,\, \frac{\mathbb P (A \, \cap \, B)}{\mathbb P (B)} $$

Nous appelons dans un premier temps la classe GaussianNB( ), puis nous faisons entrer la méthode fit(), qui va nous permettre d'entraîner notre modèle à l'aide des données splitées plus haut, et représentant nos données d'entrainement.

In [82]:
model = GaussianNB()
model.fit(x_train, y_train)

GaussianNB()

Nous allons maintenant tester notre modèle à l'aide de nos données test, et tenter de faire une prédiction.


In [83]:
y_pred = model.predict(x_test) 

print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 53.333333333333336


### Test du modèle

In [84]:
df_test = pandas.read_excel('../excel/question.xlsx', sheet_name = 'risk')

df_test = df_test.drop(columns = ["Unnamed: 0"])
df_test['Q1'] = df_test['Q1'].str.split(' - ').str[0].astype(int)

intervals = [0, 10, 18, 24, 52]
representation = [0, 1, 2, 3]
df_test['Total'] = pandas.cut(df_test['Total'], bins = intervals, labels = representation)

df_test['Total'] = df_test['Total'].astype(int)

training_columns_test = df_test.drop(columns = ["Total"])
predicted_column_test = df_test['Total']


y_pred = model.predict(training_columns_test) 
df_test['Predictions'] = y_pred

df_test["Predictions"] = df_test["Predictions"]

df_test.to_excel("Predictions_points_NaiveBays_1.xlsx")

# Modèle 2 - One-vs-the-rest

Nous commençons par appeler la classe OneVsTheRestClassifier, puis nous passons en paramètre la classe LinearSVC.

In [85]:
clf = OneVsRestClassifier(LinearSVC(random_state=0))
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test) 
from sklearn import metrics 
print("oneVsRestclassifier Classifier model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

oneVsRestclassifier Classifier model accuracy(in %): 61.66666666666667


/home/sab/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sab/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sab/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Test du modèle

In [86]:
df_test = pandas.read_excel('../excel/question.xlsx', sheet_name = 'risk')

df_test = df_test.drop(columns = ["Unnamed: 0"])
df_test['Q1'] = df_test['Q1'].str.split(' - ').str[0].astype(int)

intervals = [0, 10, 18, 24, 52]
representation = [0, 1, 2, 3]
df_test['Total'] = pandas.cut(df_test['Total'], bins = intervals, labels = representation)

df_test['Total'] = df_test['Total'].astype(int)

training_columns_test = df_test.drop(columns = ["Total"])
predicted_column_test = df_test['Total']

y_pred = model.predict(training_columns_test) 
df_test['Predictions'] = y_pred

df_test.to_excel("Predictions_points_OneVsRestClassifier_1.xlsx")

# Modèle 3 - Output-code-classifier

Nous commençons par appeler la classe OutputCodeClassifier, puis nous passons en paramètre la classe LinearSVC.

In [87]:
clf2 = OutputCodeClassifier(LinearSVC(random_state=0), code_size=2, random_state=0)
clf2.fit(x_train,y_train)

y_pred = clf2.predict(x_test) 
from sklearn import metrics 
print("Outcode Classifier model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Outcode Classifier model accuracy(in %): 70.0


/home/sab/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sab/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sab/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sab/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sab/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Test du modèle 

In [88]:
df_test = pandas.read_excel('../excel/question.xlsx', sheet_name = 'risk')

df_test = df_test.drop(columns = ["Unnamed: 0"])
df_test['Q1'] = df_test['Q1'].str.split(' - ').str[0].astype(int)

intervals = [0, 10, 18, 24, 52]
representation = [0, 1, 2, 3]
df_test['Total'] = pandas.cut(df_test['Total'], bins = intervals, labels = representation)

df_test['Total'] = df_test['Total'].astype(int)

training_columns_test = df_test.drop(columns = ["Total"])
predicted_column_test = df_test['Total']

y_pred = model.predict(training_columns_test) 
df_test['Predictions'] = y_pred

df_test.to_excel("Predictions_points_OutputCodeClassifier_1.xlsx")